In [1]:
# !pip install yfpy

In [20]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from yfpy.query import YahooFantasySportsQuery
from yahoo_oauth import OAuth2
import xml.etree.ElementTree as ET


In [ ]:
load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [18]:
def strip_namespace(tag):
    """Remove namespace from the tag name."""
    return tag.split("}")[-1] if "}" in tag else tag

def parse_element(element):
    parsed_data = {}
    children = list(element)

    # Check if all children have the same tag (implying a list)
    child_tags = [strip_namespace(child.tag) for child in children]
    is_list = len(set(child_tags)) == 1 and len(children) > 1

    # Convert child elements into dictionary keys
    if is_list:
        return [parse_element(child) for child in children]  # Return a list directly
    else:
        for child in children:
            tag = strip_namespace(child.tag)
            child_data = parse_element(child)

            # Handle multiple children with the same tag by storing them as lists
            if tag in parsed_data:
                if not isinstance(parsed_data[tag], list):
                    parsed_data[tag] = [parsed_data[tag]]
                parsed_data[tag].append(child_data)
            else:
                parsed_data[tag] = child_data

    # If the element has text content, add it
    text = element.text.strip() if element.text else ""
    if text and not parsed_data:
        return text  # Return text if no nested structure

    return parsed_data or text  # Return text if no nested structure

def xml_to_dict(xml_string):
    root = ET.fromstring(xml_string)
    return {strip_namespace(root.tag): parse_element(root)}

In [ ]:
# Use last season stats so there aren't any spoilers
current = False
league_id = "67269" if current else "97108"
game_id = 453 if current else 427

query = YahooFantasySportsQuery(
    league_id=league_id,
    game_code="nhl",
    game_id=game_id,
    yahoo_consumer_key=CLIENT_ID,
    yahoo_consumer_secret=CLIENT_SECRET
)

oauth = OAuth2(
    CLIENT_ID,
    CLIENT_SECRET,
    browser_callback=True,
)

In [5]:
BASE_URL = "https://fantasysports.yahooapis.com/fantasy/v2"

In [13]:
def api_request(url):
    if not oauth.token_is_valid():
        oauth.refresh_access_token()
    headers = {
        "Authorization": f"Bearer {oauth.access_token}",
        "Content-Type": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(response.text)
        response.raise_for_status()
    xml = response.text
    data = xml_to_dict(response.text)["fantasy_content"]
    return data

In [14]:
def get_league():
    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key};out=standings"
    resp_data = api_request(url)["league"]
    return resp_data

In [21]:
league = get_league()
teams = league["standings"]["teams"]
teams

[{'team_key': '427.l.97108.t.2',
  'team_id': '2',
  'name': 'Eric’s a Bitch Team',
  'url': 'https://hockey.fantasysports.yahoo.com/2023/hockey/97108/2',
  'team_logos': {'team_logo': {'size': 'large',
    'url': 'https://s.yimg.com/cv/apiv2/default/nhl/nhl_12_k.png'}},
  'waiver_priority': '4',
  'number_of_moves': '59',
  'number_of_trades': '0',
  'roster_adds': {'coverage_type': 'week',
   'coverage_value': '26',
   'value': '0'},
  'clinched_playoffs': '1',
  'league_scoring_type': 'headpoint',
  'draft_position': '4',
  'has_draft_grade': '0',
  'managers': {'manager': {'manager_id': '2',
    'nickname': 'Ken',
    'guid': 'TPMH3J2R4LWWFJV4GPPLVJPZYI',
    'email': 'dragonkenkj@gmail.com',
    'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg',
    'felo_score': '656',
    'felo_tier': 'silver'}},
  'team_stats': {'coverage_type': 'season',
   'season': '2023',
   'stats': [{'stat_id': '1', 'value': '304'},
    {'stat_id': '2', 'value': '478'},
    {'s

In [9]:
def get_team_name(team_key):
    team_name = next(team["name"] for team in teams if team["team_key"] == team_key)
    return team_name 

In [24]:
LEAGUE_KEY = query.get_league_key(season=None)

In [25]:
league_info = query.get_league_info()

In [26]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [27]:
team_info = query.get_league_teams()
team_info[3].clean_data_dict()['name']

b'Miami Steamrollers'

In [ ]:
LEAGUE_SETTINGS = query.get_league_settings()
SCORING_CATEGORIES = LEAGUE_SETTINGS.stat_categories.stats
for i in range(len(SCORING_CATEGORIES)):
    SCORING_CATEGORIES[i] = SCORING_CATEGORIES[i].clean_data_dict()
SCORING_CATEGORIES = pd.DataFrame(data=SCORING_CATEGORIES)

SCORING_MODIFIERS = data=LEAGUE_SETTINGS.stat_modifiers.stats
for i in range(len(SCORING_MODIFIERS)):
    SCORING_MODIFIERS[i] = SCORING_MODIFIERS[i].clean_data_dict()
SCORING_MODIFIERS = pd.DataFrame(data=SCORING_MODIFIERS)

SCORING_SETTINGS = pd.merge(left=SCORING_CATEGORIES, right=SCORING_MODIFIERS, on='stat_id', how='inner')

Draft Info

In [121]:
draft = query.get_league_draft_results()

Seperate Draft into by Team

In [122]:
draft = pd.DataFrame(data=draft, columns=['Overall', 'Round', 'Team ID', 'Player ID'])

team_drafts = {}
team_keys = []
for ID in range(1, draft['Team ID'].nunique()+1):
    team_keys = [LEAGUE_KEY + '.t.' + str(ID) for ID in range(1, draft['Team ID'].nunique()+1)] 
team_keys = tuple(team_keys)

In [123]:
team_drafts = {team_key : draft[draft['Team ID'] == team_key] for team_key in team_keys}

Assign Best and Worst Draft

In [127]:
'''
Best/Worst Draft
'''
total_draft_points = dict.fromkeys(team_keys, 0)
for team in team_drafts:
    # pull each team draft 
    draft = list(team_drafts[team]['Player ID'])
    stats = get_players(player_keys=draft)
    
    for j in range(len(draft)):
        # pull each player from the draft
        # pull each player's stats
        player_stats = stats[j].player_stats.clean_data_dict()['stats']
        
        for i in range(len(player_stats)):
            # pull league scoring settings
            # multiply league settings by player's stats
            stat = player_stats[i]['stat'].clean_data_dict()
            if stat['stat_id'] in list(SCORING_SETTINGS['stat_id']):
                total_draft_points[team] += stat['value'] * list(SCORING_SETTINGS['value'])[list(SCORING_SETTINGS['stat_id']).index(stat['stat_id'])]

total_draft_points = {key : round(value, 1) for key, value in total_draft_points.items()}
total_draft_points = dict(sorted(total_draft_points.items(), key=lambda item: item[1], reverse=True))
total_draft_points = {key : (get_team_name(key), value) for key, value in total_draft_points.items()}

print('Best Drafts:')
print(f'1. {list(total_draft_points.values())[0][0]} ({str(list(total_draft_points.values())[0][1])})')
print(f'2. {list(total_draft_points.values())[1][0]} ({str(list(total_draft_points.values())[1][1])})')
print(f'3. {list(total_draft_points.values())[2][0]} ({str(list(total_draft_points.values())[2][1])})\n')
print('Worst Drafts:')
print(f'1. {list(total_draft_points.values())[len(total_draft_points)-1][0]} ({str(list(total_draft_points.values())[len(total_draft_points)-1][1])})')
print(f'2. {list(total_draft_points.values())[len(total_draft_points)-2][0]} ({str(list(total_draft_points.values())[len(total_draft_points)-2][1])})')
print(f'3. {list(total_draft_points.values())[len(total_draft_points)-3][0]} ({str(list(total_draft_points.values())[len(total_draft_points)-3][1])})')

Best Drafts:
1. Tim's Terrific Team (5554.2)
2. Theodore's Poo Poo Hospital (5315.4)
3. Eric’s a Bitch Team (5312.0)

Worst Drafts:
1. Miami Steamrollers (4597.7)
2. Kevin's Incredible Team (4675.8)
3. Cyrus's Cursed Team (4866.1)


Playground

In [ ]:
# Mcdavid = query.get_player_stats_for_season("453.p.6743")

In [ ]:
# Mcdavid.stats

In [ ]:
query.get_league_matchups_by_week(18)

[Matchup({
   "is_consolation": 0,
   "is_playoffs": 0,
   "status": "preevent",
   "teams": [
     {
       "team": {
         "draft_position": 1,
         "has_draft_grade": 0,
         "is_owned_by_current_login": 1,
         "league_scoring_type": "headpoint",
         "managers": {
           "manager": {
             "email": "amtadam14@hotmail.ca",
             "felo_score": 571,
             "felo_tier": "bronze",
             "guid": "NGSXRXJK2U3UJVTCC2PO3L2A6M",
             "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
             "is_commissioner": 0,
             "is_current_login": 1,
             "manager_id": 1,
             "nickname": "adam"
           }
         },
         "name": "McFuck",
         "number_of_moves": 38,
         "number_of_trades": 0,
         "roster_adds": {
           "coverage_type": "week",
           "coverage_value": 17,
           "value": 0
         },
         "team_id": 1,
         "team_key": "453.l.6

In [ ]:
transactions = query.get_league_transactions()

In [ ]:
adds = {}
for transaction in transactions:
    if transaction.type == "drop":
        print(transaction.players)

[Player({
  "display_position": "D",
  "editorial_team_abbr": "CBJ",
  "name": {
    "ascii_first": "Ivan",
    "ascii_last": "Provorov",
    "first": "Ivan",
    "full": "Ivan Provorov",
    "last": "Provorov"
  },
  "player_id": 6749,
  "player_key": "453.p.6749",
  "position_type": "P",
  "transaction_data": {
    "destination_type": "waivers",
    "source_team_key": "453.l.67269.t.4",
    "source_team_name": "(",
    "source_type": "team",
    "type": "drop"
  }
})]
[Player({
  "display_position": "C,RW",
  "editorial_team_abbr": "UTA",
  "name": {
    "ascii_first": "Nick",
    "ascii_last": "Schmaltz",
    "first": "Nick",
    "full": "Nick Schmaltz",
    "last": "Schmaltz"
  },
  "player_id": 6386,
  "player_key": "453.p.6386",
  "position_type": "P",
  "transaction_data": {
    "destination_type": "waivers",
    "source_team_key": "453.l.67269.t.4",
    "source_team_name": "(",
    "source_type": "team",
    "type": "drop"
  }
})]
[Player({
  "display_position": "C",
  "editori

In [ ]:
print(transactions[1])

Transaction({
  "players": {
    "player": {
      "display_position": "C,RW",
      "editorial_team_abbr": "UTA",
      "name": {
        "ascii_first": "Nick",
        "ascii_last": "Schmaltz",
        "first": "Nick",
        "full": "Nick Schmaltz",
        "last": "Schmaltz"
      },
      "player_id": 6386,
      "player_key": "453.p.6386",
      "position_type": "P",
      "transaction_data": {
        "destination_type": "waivers",
        "source_team_key": "453.l.67269.t.4",
        "source_team_name": "(",
        "source_type": "team",
        "type": "drop"
      }
    }
  },
  "status": "successful",
  "timestamp": 1739232690,
  "transaction_id": 525,
  "transaction_key": "453.l.67269.tr.525",
  "type": "drop"
})
